## **About this Script**

This script creates a convolutional neural network (CNN) for DNA sequence classification. 

Specifically, it classifies promoter sequences into positive (+) or negative (-) classes. This type of classification can have a variety of applications. For instance, researchers may classify sequences containing regions that wrap around histones as positive and sequences that don't as negative. Such classification would enable prediction of histone profiles from sequences and aid in understanding gene expression patterns. Here is an example of a paper that applied a CNN for such classification: https://www.scirp.org/journal/paperinformation.aspx?paperid=65923

**A promoter**: a DNA sequence that turns a gene on or off. 

This script was adapted and modified from https://github.com/ahsanazim/cnn
If you have any questions, please contact: maese005@umn.edu

## **Step 1:** Get Data

First, mount your Google Drive to your Google Colab.

This will allow you to share files and access images stored in Google Drive from Google Collab.

Once you run this code, you will receive a prompt asking you to click a link. When you click on this link, it will allow for 'Google Files Stream' to access your drive.

After, you will receive a long authentication code which you need to copy and enter into your Colab's notebook.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Now that your Google Collab is mounted to your Google Drive, you must obtain the data for this script.

The data can be obtained at this link: https://drive.google.com/file/d/1xsRPoHSNQv5r9gK5uwVfVmY_qjBtRaIj/view?usp=sharing

To access the data from your Google Drive, click on the link. Save the data to your Google Drive. 

## **Step 2:** Import python libraries and define functions that will ease development of our CNN

In [ ]:
def conv_labels(labels, dataset='splice'):
    converted = []
    for label in labels:
        if dataset == 'splice':
            if label == 'EI':
                converted.append(0)
            elif label == 'IE':
                converted.append(1)
            elif label == 'N':
                converted.append(2)
        elif dataset == 'promoter':
            if label =='+':
                converted.append(0)
            elif label == '-':
                converted.append(1)
    return converted

def create_dict(nucleotides):
    vec_dict = {}
    perms = k_len_perms(nucleotides, 3)
    perms.sort()
    for idx, seq in enumerate(perms):
        hot_vec = [ 0 for i in range(0, len(perms))]
        hot_vec[idx] = 1
        vec_dict[seq] = hot_vec
    return vec_dict

def k_len_perms(letters, k):
    n = len(letters)
    perms = []
    k_len_perms_hlpr(perms, letters, "", n, k)
    return perms

def k_len_perms_hlpr(perms, letters, prefix, n, k):
    if (k == 0):
        perms.append(prefix)
        return
    for i in range(0, n):
        newPrefix = prefix + letters[i]
        k_len_perms_hlpr(perms, letters, newPrefix, n, k - 1)

def get_rep_mat(seq, hot_vec_dict, k=3, r_size=2):
    words = word_seq(seq, k)
    rep_mat = create_rep_mat(words, hot_vec_dict, r_size)
    return rep_mat

def word_seq(seq, k, stride=1):
    i = 0
    words = []
    while i <= len(seq) - k:
        words.append(seq[i: i + k])
        i += stride
    return words

def create_rep_mat(words, hot_vec_dict, r_size):
    mat_len = len(words) - r_size + 1
    mat = [[] for i in range(0, mat_len)]
    i = 0
    while i < mat_len:
        j = i
        while j < i + r_size:
            mat[i].append(hot_vec_dict[words[j]])
            j += 1
        i += 1
    return mat

def get_rep_mats(seqs):
    rep_mats = []
    hot_vec_dict = create_dict('ACGT')
    for seq in seqs:
        rep_mat = get_rep_mat(seq, hot_vec_dict, k=3, r_size=1)
        rep_mats.append(rep_mat)
    return rep_mats

We will be using **Keras** to build our CNN.

**Keras** is a deep learning Python API of TensorFlow (which is a platform used to solve machine learning problems). The core data structures of Keras are layers and models. The simplest model is the **Sequential model**, which can be described as a linear stack of layers. 

Additional information about Keras here: https://keras.io/about/#:~:text=Keras%20is%20a%20deep%20learning,key%20to%20doing%20good%20research.

In [ ]:
#numpy is helpful when we want to work with numbers and numerical operations. 
import numpy as np

np.random.seed(123)  #Whenever you build a model, always set the seed for reproducibility purposes. 

#We will be using Keras to build our CNN. 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D #These are the layers we will be using in our model. 
from keras.utils import np_utils
from keras.datasets import mnist

## **Step 3:** Prepare training and testing data


Read the promoter sequence data text file into Python. 

First, we will use the open() function, which will open the text file for reading. 

Second, we will read text from the file. 

Lastly, we will close the file using the file close() method. 

In [ ]:
f=open("/content/drive/My Drive/AI_Workshop/Code and Exercises/promoters.data.txt")

seqs = []
labels = []
for line in f: #This for loop will fill the seqs and labels list. 
  line_no_wspace = line.replace(" ","")
  line_no_nwline = line_no_wspace.replace("\n","")
  line_arr = line_no_nwline.split(",")
  label = line_arr[0]
  seq = line_arr[2]
  
  #Sequence cleaning
  seq = seq.upper()    
  seq = seq.replace("\t","")      
  seq = seq.replace("N","A")  
  seq = seq.replace("D","G")
  seq = seq.replace("S","C")
  seq = seq.replace("R","G")

  labels.append(label)
  seqs.append(seq)

f.close()

In [ ]:
seqs #Print the sequences stored in this file. It's always a good idea to take a look at our data!

In [ ]:
labels #Print the labels for each sequence. 

Explore the sequence data.
*   What data structure is it? (It's a list).
*   What are the dimensions? (Its length is 106, so it contains 106 sequences).



In [ ]:
type(seqs) #list
len(seqs) #106 sequences. 
seqs[0] #Lets take a look at the first sequence in our sequence list, which is located at index 0. 

'TACTAGCAATACGCTTGCGTTCGGTGGTTAAGTATGTATAATGCGCGGGCTTGTCGT'

Convert the sequence data into an array of representation matrices using the function defined above **get_rep_mats** 

In [ ]:
X=get_rep_mats(seqs) 

y=labels #Set the labels as y. 

Explore the sequence data now that it has been converted from letters (images of nucleotides) to numbers. 
*   What data structure is it? (It's still a list).
*   What are the dimensions? (Its length is still 106 because it contains 106 sequences as before). 
*   What did the get_rep_mats function do? (It converted each letter, in each sequence, to a vector of 1's and 0's). 


In [ ]:
type(X) #list
len(X) #106
len(X[0]) #55...55 sets of 1's and 0's. Each set represents a letter in sequence 1. 
len(X[0][0]) #1...a vector of a bunch of 1's and 0's that represent the first letter in the first sequence.

1

Split data into training and testing data.

The training data will have 90 images, the testing will have 16 (recall there are 106 total). 

In [ ]:
for i in X:
    for idx, j in enumerate(i):
        i[idx] = j[0]

y = conv_labels(y, "promoter") #Convert to integer labels
X = np.asarray(X) #Work with np arrays
y = np.asarray(y)
X_train = X[0:90]
X_test = X[90:]
y_train = y[0:90]
y_test = y[90:]

In [ ]:
X_train.shape #90, 55, 64

(90, 55, 64)

In [ ]:
y_train

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

## **Step 4:** Preprocess the data into forms that will work for the CNN. 

Here, it's very important that we are aware of our dimensions. 

In [ ]:
#Preprocess input data
X_train = X_train.reshape(X_train.shape[0], 55, 64, 1)  # (90, 55, 64) --> (90, 55, 64, 1)...reshape the data by adding 4th dimensions. 55 image rows, 65 image columns. 
X_test = X_test.reshape(X_test.shape[0], 55, 64, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

#Convert 1-dimensional class arrays to 3-dimensional class matrices.
Y_train = np_utils.to_categorical(y_train, 2)
Y_test = np_utils.to_categorical(y_test, 2)

In [ ]:
X_train.shape #90, 55, 64, 1 because we have 90 images of 55 by 64 pixels not in RGM (3 color channels). We are using grayscale, so the color channel is 1. 

(90, 55, 64, 1)

In [ ]:
len(X_train) #90 training DNA sequences

90

In [ ]:
len(y_train) #90 training labels for DNA sequences

90

In [ ]:
y_train #We have a binary label of 1's and 0s'. 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

## **Step 5:** Build the neural network by adding multiple layers. 

We will do this by creating a sequential model and adding the layers. 

**What is an input shape tensor?**

**Tensors**: matrices with shapes that flow between layers. 

In Keras, the input layer itself is not a layer, it is a tensor. It's the starting tensor that you send to the first hidden layer. This tensor must have the same shape as your training data. 

Ex: if you have 90 images of 55 x 64 pixels in RGB (3 color channels), the shape of your input data is (90, 55, 64, 3). Therefore, when you use **input_shape** in keras, you must use this shape. We will use 55, 64, 1 because we are in the grayscale

Your input shape is the only shape you have to define or provide in your model because your model can't know it...only you know it based on your training data. All other shapes are calculated automatically based on the units and particularities of each layer. 

**What is a convolution?**

A convolution/weight matrix/convolution matrix/kernel/filter is a small matrix of weights with the same number of rows as there are columns. 

A 3x3 convolution filter is generally used when you apply 2D convolutions on images. There's always a third dimension, but you don't have to include it; it is included automatically. This third dimension indicates the number of channels of the input image. Ex: a gray scale image requires 3x3x1...RGB image requires 3x3x3. **A convolution filter is referred to by its first 2 dimensions**

More information here: https://medium.com/@icecreamlabs/3x3-convolution-filters-a-popular-choice-75ab1c8b4da8#:~:text=It%20is%20used%20for%20blurring,a%20kernel%20and%20an%20image.&text=While%20applying%202D%20convolutions%20like,a%20third%20dimension%20in%20size.

In [ ]:
model = Sequential() #Initiate the sequential model. 

#Apply a 3x3 convolution with 64 output filters on a 55 x 64 image in the grayscale. 
#Note: when using this as the first layer in a model, provide the keyword argument input_shape (Ex: input_shape=(130, 130, 3) for 130x130 RGB images). 
model.add(Convolution2D(64, 3, 3, activation='relu', input_shape=(55, 64, 1))) 
#input_shape must contain 3 dimensions. Internally, Keras will add the batch dimensions, making it 4 dimensions total. 
#The 1 indicates the channel. It lets Keras know that you are working on a grayscale image (because it is 1). 

model.add(Convolution2D(64, 3, 3, activation='relu')) 
#The kernel_size argument (3,3) represents height, width of the kernel. Kernel depth will be the same as the depth of the image..
#64 is the filter parameter in Keras's Convolution2D function.
#There is no good answer for the question, how do you determine the value of this filter parameter?
#You would have to carefully design and fine tune your model through many experiments. 
#However, there are some guidelines as to how to choose the number of filters. 
#https://stackoverflow.com/questions/48243360/how-to-determine-the-filter-parameter-in-the-keras-conv2d-function

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))   # since 2 classes

Compile the model. 

You must compile the model before you can train or evaluate it. 

Here, the efficient **ADAM optimization** algorithm is used. 

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
#Here, the loss function chosen is sparse_categorical_crossentropy. 
#This loss function is generally recommended when you have a 1D integer encoded target.

Fit the model. 

Call fit() to train the model by slicing the data into batches of size batch_size and repeatedly iterating over the entire dataset for the number of epochs specified. 

In [ ]:
model.fit(x=X_train,
          y=Y_train,
          validation_data=(X_test, Y_test), 
          batch_size=32,
          epochs=20,
          verbose=1)

Epoch 1/20
3/3 [==============================] - 1s 117ms/step - loss: 0.6895 - accuracy: 0.5891 - val_loss: 0.7429 - val_accuracy: 0.0000e+00
Epoch 2/20
3/3 [==============================] - 0s 35ms/step - loss: 0.6752 - accuracy: 0.6047 - val_loss: 0.8248 - val_accuracy: 0.0000e+00
Epoch 3/20
3/3 [==============================] - 0s 36ms/step - loss: 0.6538 - accuracy: 0.5952 - val_loss: 0.9039 - val_accuracy: 0.0000e+00
Epoch 4/20
3/3 [==============================] - 0s 35ms/step - loss: 0.6545 - accuracy: 0.5835 - val_loss: 0.9346 - val_accuracy: 0.0000e+00
Epoch 5/20
3/3 [==============================] - 0s 37ms/step - loss: 0.6399 - accuracy: 0.5874 - val_loss: 0.9122 - val_accuracy: 0.0000e+00
Epoch 6/20
3/3 [==============================] - 0s 39ms/step - loss: 0.6305 - accuracy: 0.5601 - val_loss: 0.8763 - val_accuracy: 0.0000e+00
Epoch 7/20
3/3 [==============================] - 0s 43ms/step - loss: 0.5920 - accuracy: 0.6403 - val_loss: 0.8741 - val_accuracy: 0.0000e+0

In [ ]:
model.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 18, 21, 64)        640       
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 6, 7, 64)          36928     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 3, 3, 64)          0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 3, 3, 64)          0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 576)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 128)               73856     
_________________________________________________________________
dropout_27 (Dropout)         (None, 128)             

Estimate the performance of the model on unseen data. 

In [ ]:
model.evaluate(X_test, Y_test, verbose=0)

[0.9265512228012085, 0.4375]

In [ ]:
history=model.fit(X_train, Y_train, batch_size=32, epochs=20, validation_data=(X_test, Y_test))

Epoch 1/20
3/3 [==============================] - 0s 69ms/step - loss: 0.0471 - accuracy: 1.0000 - val_loss: 0.8365 - val_accuracy: 0.5625
Epoch 2/20
3/3 [==============================] - 0s 41ms/step - loss: 0.0348 - accuracy: 1.0000 - val_loss: 0.9376 - val_accuracy: 0.5000
Epoch 3/20
3/3 [==============================] - 0s 41ms/step - loss: 0.0291 - accuracy: 1.0000 - val_loss: 1.1854 - val_accuracy: 0.4375
Epoch 4/20
3/3 [==============================] - 0s 41ms/step - loss: 0.0179 - accuracy: 1.0000 - val_loss: 1.2927 - val_accuracy: 0.3750
Epoch 5/20
3/3 [==============================] - 0s 47ms/step - loss: 0.0214 - accuracy: 1.0000 - val_loss: 1.0377 - val_accuracy: 0.4375
Epoch 6/20
3/3 [==============================] - 0s 41ms/step - loss: 0.0166 - accuracy: 1.0000 - val_loss: 0.8378 - val_accuracy: 0.6875
Epoch 7/20
3/3 [==============================] - 0s 42ms/step - loss: 0.0149 - accuracy: 1.0000 - val_loss: 0.8975 - val_accuracy: 0.6250
Epoch 8/20
3/3 [===========

The returned history object contains a record of the loss values and metric values acquired during training. 

In [ ]:
history.history

{'accuracy': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'loss': [0.04711845889687538,
  0.03484797105193138,
  0.029053818434476852,
  0.017926031723618507,
  0.021369624882936478,
  0.01657998003065586,
  0.014896935783326626,
  0.009546159766614437,
  0.009708361700177193,
  0.006332335993647575,
  0.00859782937914133,
  0.006212372332811356,
  0.004263704176992178,
  0.0024531022645533085,
  0.00557808019220829,
  0.003262109821662307,
  0.002391067799180746,
  0.004019764252007008,
  0.0019499287009239197,
  0.0016564119141548872],
 'val_accuracy': [0.5625,
  0.5,
  0.4375,
  0.375,
  0.4375,
  0.6875,
  0.625,
  0.5,
  0.4375,
  0.4375,
  0.4375,
  0.4375,
  0.5,
  0.5625,
  0.5625,
  0.5625,
  0.625,
  0.5,
  0.5,
  0.5],
 'val_loss': [0.8364540934562683,
  0.9376440048217773,
  1.1853907108306885,
  1.2927249670028687,
  1.0376622676849365,
  0.8378043174743652,
  0.8974912166595459